In [17]:
import xml.etree.ElementTree
import cv2
import glob
import os
import numpy as np
from openslide import OpenSlide

xml_list = sorted(glob.glob(
    '/media/nathan/DATA/histo-seg-data/slide_for_testing_SPIE_2017/*xml'))
svs_list = sorted(glob.glob(
    '/media/nathan/DATA/histo-seg-data/slide_for_testing_SPIE_2017/*svs'))

%matplotlib inline
from matplotlib import pyplot as plt

In [18]:
GradeCode = {
    '3+3': 0,
    '4+4': 1,
    '4+5': 2,
    'BN':  3,
    'ST':  4
}

GradeColors = [
    [242,84,94],
    [20,20,20],
    [52,35,166],
    [91,92,235],
    [91,146,121],
]

In [20]:
outpath = '/home/nathan/histo-seg/semantic-pca/data/wsi_annotation'

index=1
for xmlpath, svspath in zip(xml_list, svs_list):
#     xmlpath = xml_list[-1]
#     svspath = svs_list[-1]

    print index,
    print os.path.basename(xmlpath)
    print os.path.basename(svspath)
    e = xml.etree.ElementTree.parse(xmlpath).getroot()
    svsimg = OpenSlide(svspath)

    if svsimg.level_count == 4:
        svslevel = 2
        divf = 16
    else:
        svslevel = 2
        divf = 16

    regions = e.findall('Annotation')[0].findall('Regions')[0].findall('Region')

    contours = []
    grades = []
    for region in regions:
        grade = region.attrib['Text']
        grades.append(GradeCode[grade])
        area = region.attrib['AreaMicrons']
        vertices = region.findall('Vertices')[0].findall('Vertex')
        contours.append(np.asarray([(int(vert.attrib['X'])/divf, int(vert.attrib['Y'])/divf) 
                         for vert in vertices], dtype=np.int32))
        print '\tGrade: {} Area: {}'.format(
            grade, area)

    try:
        img = svsimg.read_region((0,0), svslevel, svsimg.level_dimensions[svslevel])
        img = np.asarray(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        mask = np.zeros(svsimg.level_dimensions[svslevel][::-1], dtype=np.uint8)+255
    except:
        print 'FAILED SVS LOAD'
    #     ?continue

    # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    for cnt, grd in zip(contours, grades):
        cv2.drawContours(img, [cnt], 0, GradeColors[grd], 7)
        cv2.drawContours(mask, [cnt], 0, (grd), -1)

    maskname = os.path.basename(xmlpath).replace('xml', 'png')
    imgname = os.path.basename(xmlpath).replace('xml', 'jpg')
    print '\t', img.shape
    print '\t'+outname
    cv2.imwrite(os.path.join(outpath, maskname), mask)
    cv2.imwrite(os.path.join(outpath, imgname), img[:,:,::-1])

    index+=1

#     print img.shape, img.min(), img.max()
#     fig, ax = plt.subplots(1,1, figsize=(7,9))
#     ax.matshow(img, cmap='gray')


1 1304700.xml
1304700.svs
	Grade: 3+3 Area: 9193710.7
	Grade: BN Area: 8999604.8
	(1640, 2988, 3)
	1305489.png
2 1304701.xml
1304701.svs
	Grade: 3+3 Area: 2721921.7
	Grade: ST Area: 11595906.4
	Grade: BN Area: 39266167.6
	(2090, 2988, 3)
	1305489.png
3 1304702.xml
1304702.svs
	Grade: 3+3 Area: 8763987.0
	Grade: BN Area: 62313971.9
	(2074, 3735, 3)
	1305489.png
4 1304703.xml
1304703.svs
	Grade: 3+3 Area: 913782.1
	Grade: BN Area: 91255327.8
	(2287, 3610, 3)
	1305489.png
5 1304704.xml
1304704.svs
	Grade: 3+3 Area: 12318877.4
	Grade: BN Area: 61738668.0
	(2782, 3112, 3)
	1305489.png
6 1304705.xml
1304705.svs
	Grade: 3+3 Area: 10277622.7
	Grade: BN Area: 27062909.4
	(2749, 2739, 3)
	1305489.png
7 1304706.xml
1304706.svs
	Grade: 3+3 Area: 1904269.5
	Grade: 3+3 Area: 6066636.0
	Grade: BN Area: 11266830.5
	(2408, 4108, 3)
	1305489.png
8 1304709.xml
1304709.svs
	Grade: 3+3 Area: 12064326.3
	Grade: BN Area: 12064367.7
	(2828, 3859, 3)
	1305489.png
9 1304711.xml
1304711.svs
	Grade: 3+3 Area: 618

In [61]:
svsimg = OpenSlide('/media/nathan/DATA/histo-seg-data/slide_for_testing_SPIE_2017/1305492.svs')
for key in svsimg.properties.iterkeys():
    print key, svsimg.properties[key]

aperio.AppMag 20
aperio.Date 02/07/17
aperio.DisplayColor 0
aperio.Exposure Scale 0.000001
aperio.Exposure Time 109
aperio.Filename MaZ-045
aperio.Focus Offset 0.000000
aperio.ICC Profile ScanScope v1
aperio.ImageID 1015366
aperio.Left 25.546640
aperio.LineAreaXOffset 0.013011
aperio.LineAreaYOffset -0.012034
aperio.LineCameraSkew 0.000000
aperio.MPP 0.4910
aperio.OriginalHeight 39469
aperio.OriginalWidth 42672
aperio.Parmset IHC
aperio.ScanScope ID SS1562
aperio.SessonMode NR
aperio.StripeWidth 2032
aperio.Time 19:41:48
aperio.Time Zone GMT-08:00
aperio.Top 21.369835
aperio.User 06f562cb-e5d4-46ac-b0a9-cbc9c5a6af18
openslide.comment Aperio Image Library v12.0.15 
42672x39469 [0,100 41832x39369] (240x240) JPEG/YCC Q=70 (91)|AppMag = 20|StripeWidth = 2032|ScanScope ID = SS1562|Filename = MaZ-045|Date = 02/07/17|Time = 19:41:48|Time Zone = GMT-08:00|User = 06f562cb-e5d4-46ac-b0a9-cbc9c5a6af18|Parmset = IHC|MPP = 0.4910|Left = 25.546640|Top = 21.369835|LineCameraSkew = 0.000000|LineAreaXO

In [120]:
help(cv2.drawContours)

Help on built-in function drawContours in module cv2:

drawContours(...)
    drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]) -> None



In [124]:
xy_np = np.asarray(XY)

In [15]:
ops = OpenSlide('/media/labshare/_PROSTATE_CANCER_PROJECT_/_WSI_/Deep_Learning_image_data/_TEST_SLIDES/4+5/WSI_not_used_to_extract_training_tiles/1305490.svs')
img = ops.read_region((0,0), 2, ops.level_dimensions[2])